# 3.1 Rules as Views

One way to see rules is to think of them as creating views.

All rules can be thought of in this way but some can be used to create more traditional views.

Views simplify SPARQL, reduce repetition in code, and accelerate query performance - they are incredible useful.

## Example

With the rule below, we create a view of the profit from our best selling products - that is, the products that have sold over 1000 units.

In [45]:
view_data = """
@prefix : <https://rdfox.com/example#> .

:productA a :Product ;
    :unitsSold 5000 ;
    :hasCost 9 ;
    :hasSalePrice 10 .

:productB a :Product ;
    :unitsSold 2000 ;
    :hasCost 80 ;
    :hasSalePrice 100 .

:productC a :Product ;
    :unitsSold 10 ;
    :hasCost 30 ;
    :hasSalePrice 50 .

"""

In [46]:
view_rules = """
@prefix : <https://rdfox.com/example#> .

[?product, a, :highPerformingProduct],
[?product, :hasTotalProfit, ?totalProfit] :-
    [?product, a, :Product],
    [?product, :unitsSold, ?units],
    [?product, :hasCost, ?cost],
    [?product, :hasSalePrice, ?price],
    BIND ( (?price - ?cost) * ?units AS ?totalProfit ),
    FILTER (?units > 1000).

"""

In [47]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=view_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=view_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Get and issue select query
with open("../queries/3_1-RulesAsViewsQuery.rq", "r") as file:
    view_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": view_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== High Performing Products ===')
print(response.text)


=== High Performing Products ===
?product	?totalProfit	?units
<https://rdfox.com/example#productB>	40000	2000
<https://rdfox.com/example#productA>	5000	5000



## Incremental Reasoning

Adding to this, reasoning updates incrementally meaning as new data comes in or as old is removed, the view is always kept consistent with changes to the data store in real-time. This is a very efficient process which considers the smallest necessary part of the graph to compute the change.

In [48]:
new_data = """
@prefix : <https://rdfox.com/example#> .

:productD a :Product ;
    :unitsSold 10000 ;
    :hasCost 10 ;
    :hasSalePrice 110 .

"""

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=new_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and issue select query
with open("../queries/3_1-RulesAsViewsQuery.rq", "r") as file:
    view_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": view_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== High Performing Products ===')
print(response.text)


=== High Performing Products ===
?product	?totalProfit	?units
<https://rdfox.com/example#productD>	1000000	10000
<https://rdfox.com/example#productB>	40000	2000
<https://rdfox.com/example#productA>	5000	5000



## Explainable results

RDFox keeps track of the inferences it makes, storing the chain of rules that support each facts existence.

This means all inferred results can be explained in proof trees of the facts and rules that lead to them.

### Exploring explanations

Head to the **Explain** tab of the RDFox console to start visualising an explanation, or right click on an edge in the **Explore** tab and selecting **Explain**.

[Here is a link](http://localhost:12110/console/datastores/explain?datastore=default&fact=%5B%3Chttps%3A%2F%2Frdfox.com%2Fexample%23productA%3E%2C%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23type%3E%2C%20%3Chttps%3A%2F%2Frdfox.com%2Fexample%23highPerformingProduct%3E%5D) to the explanation of `productA a :highPerformingProduct`.

## Exercise

Complete the rule `3_1-RulesAsViewsRules.dlog` in the `rules` folder so that the query below return a list of product categories and the **average** units sold for all products in that category.

### Hits & helpful resources

Averages are a form of aggregates.

In [41]:
view_sparql = """
SELECT ?productCategory ?averageCategoryUnitsSold
WHERE {
    ?productCategory :hasAverageUnitsSold ?averageCategoryUnitsSold .
} ORDER BY DESC (?averageCategoryUnitsSold)
"""

Here is a representative sample of the data in `3_1-RulesAsViewsData.ttl`.

In [38]:
sample_data = """
@prefix : <https://rdfox.com/example#> .

:product0001 a :Product,
                :Chair ;
    :unitsSold 3000 ;
    :hasCost 40 ;
    :hasSalePrice 50 .

:product0002 a :Product,
                :Sofa ;
    :unitsSold 100 ;
    :hasCost 120 ;
    :hasSalePrice 300 .

"""

### Check your work

Run the query below to verify the results.

In [44]:
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get and add data
with open("../data/3_1-RulesAsViewsData.ttl", "r") as file:
    data = file.read()
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and add rules
with open("../rules/3_1-RulesAsViewsRules.dlog", "r") as rule_file:
    datalog_rule = rule_file.read()
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)
assert_response_ok(response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": view_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Product Category Average Profit ===')
print(response.text)


=== Product Category Average Profit ===
?productCategory	?averageCategoryUnitsSold
<https://rdfox.com/example/Chair>	9943.57894736842105
<https://rdfox.com/example/Product>	4939.885
<https://rdfox.com/example/Sofa>	4879.032
<https://rdfox.com/example/Stool>	2531.395061728395062
<https://rdfox.com/example/Beanbag>	2495.9



### You should see...

=== Product Category Average Profit ===
|?productCategory|?averageCategoryUnitsSold|
|-----------|-------------|
|<https://rdfox.com/example/Chair>|	9943.57894736842105|
|<https://rdfox.com/example/Product>|	4939.885|
|<https://rdfox.com/example/Sofa>|	4879.032|
|<https://rdfox.com/example/Stool>|	2531.395061728395062|
|<https://rdfox.com/example/Beanbag>|	2495.9|